In [1]:
# 기본 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 경고 뜨지 않게
import warnings
warnings.filterwarnings('ignore')

#저장라이브러리
import pickle

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 표준화
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# LabelEncoder
from sklearn.preprocessing import LabelEncoder

# 데이터를 학습용과 검증용으로 나눈다.
from sklearn.model_selection import train_test_split

import os


# 딥러닝
import tensorflow as tf

# 딥러닝 모델 구조를 정의하는 것
from tensorflow.keras.models import Sequential
# 층구조를 정의하는 것
from tensorflow.keras.layers import Dense
# 활성화 함수를 정의하는 것
from tensorflow.keras.layers import Activation

# 다중 분류를 위한 원핫 인코딩
from tensorflow.keras.utils import to_categorical

# 저장된 학습 모델 복원
from tensorflow.keras.models import load_model

# epoch마다 모델을 저장하는 함수
from tensorflow.keras.callbacks import ModelCheckpoint

# 출력한 것을 청소하는 함수
from IPython.display import clear_output
# 시간 관련
import time

gpus = tf.config.experimental.list_physical_devices('GPU')
# gpu가 있다면..
if len(gpus) > 0 :
    try :
        for gpu in gpus :
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e :
        print(e)


In [3]:
# 랜덤시드를 설정한다.
np.random.seed(1)
tf.random.set_seed(1)

In [5]:
# 데이터를 불러온다.
df1 = pd.read_csv('data/wine.csv', header=None)
df1

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,0
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,0
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,0
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,0


In [7]:
# 입력과 결과로 나눈다.
X = df1.drop(12, axis=1)
y = df1[12]

display(X)
display(y)

,0,1,2,3,4,5,6,7,8,9,10,11
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


0       1
1       1
2       1
3       1
4       1
       ..
6492    0
6493    0
6494    0
6495    0
6496    0
Name: 12, Length: 6497, dtype: int64

In [9]:
# 모델 설정
model = Sequential()

model.add(Dense(30, input_dim=12))
model.add(Activation('relu'))

model.add(Dense(12))
model.add(Activation('relu'))

model.add(Dense(8))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

In [11]:
# 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam', 
              metrics=['accuracy'])

In [13]:
# 학습 모델들이 저장될 위치
# 만약 매번 다른 파일로 저장하게 하겠다면 파일명이 매번 달라지게 해줘야 한다.
# 저장하는 파일명에 모니터링 할 수 있는 수치값이 대입되도록 설정할 수 있다.
# epoch : 학습 횟수
# loss : 학습용 데이터의 오차
# val_loss : 검증용 데이터의 오차
path1 = 'models/18_와인분류 - 자동저장/{epoch}-{val_loss}.h5'
path2 = 'models/18_와인분류 - 자동저장/최종.h5'

# 자동 저장 콜백
# save_best_only : True를 넣어주면 모니터링하는 값이 개선되었을 경우에만 저장하낟.
# 기본은 False이며 False시 매번 저장을 한다.
# monitor : 모니터링할 평가 지표
call1 = ModelCheckpoint(filepath=path1, monitor='val_loss', save_best_only=True)
call2 = ModelCheckpoint(filepath=path2, monitor='val_loss', save_best_only=True)

In [15]:
# 학습한다.
# callbacks에 사용하고 싶은 콜백들을 지정한다.
# 이 콜백들은 학습 한번이 끝나면 지정한 순서대로 동작을 한다.
model.fit(X, y, epochs=200, batch_size=500, validation_split=0.2, 
         callbacks=[call1, call2])

Epoch 1/200
 1/11 ━━━━━━━━━━━━━━━━━━━━ 29s 3s/step - accuracy: 0.7000 - loss: 1.9142

11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 64ms/step - accuracy: 0.6968 - loss: 1.6850 - val_accuracy: 1.0000 - val_loss: 0.0036
Epoch 2/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.6968 - loss: 0.8124 - val_accuracy: 1.0000 - val_loss: 0.1155
Epoch 3/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.6967 - loss: 0.4795 - val_accuracy: 0.9985 - val_loss: 0.1732
Epoch 4/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.7022 - loss: 0.4157 - val_accuracy: 0.9985 - val_loss: 0.0874
Epoch 5/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.7123 - loss: 0.3989 - val_accuracy: 0.9962 - val_loss: 0.1399
Epoch 6/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.7243 - loss: 0.3839 - val_accuracy: 0.9938 - val_loss: 0.1197
Epoch 7/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.7777 - loss: 0.3703 - val_accuracy: 0.9892 - val_loss: 0.1276
Epoch 8/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8229 - loss: 0.3621 - val_accuracy: 0.9885 - val_l

In [17]:
# 학습 모델을 불러온다.
best_model = load_model('models/18_와인분류 - 자동저장/최종.h5')
a1 = best_model.evaluate(X, y)
print(f'손실률 : {a1[0]}')
print(f'정확도 : {a1[1]}')

204/204 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4140 - loss: 1.9281
손실률 : 0.7931615710258484
정확도 : 0.7538864016532898


In [27]:
# 모델 설정
model = Sequential()

model.add(Dense(30, input_dim=12))
model.add(Activation('relu'))

model.add(Dense(22))
model.add(Activation('relu'))

model.add(Dense(14))
model.add(Activation('relu'))

model.add(Dense(6))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))